In [37]:
import pandas as pd

In [38]:
excel_dbfs_path = f"./david_finance.xlsx"
sheet_name = f"nueva_deuda_a_plazos"

In [39]:
dfp = pd.read_excel(excel_dbfs_path, engine = "openpyxl", sheet_name = sheet_name, dtype = {"monto_total_deuda":'float'})

In [40]:
tdc_df = pd.read_excel(excel_dbfs_path, engine = "openpyxl", sheet_name = "tarjetas_de_credito")

In [42]:
class InstallmentsCreator:
    def __init__(self, df: pd.DataFrame):
        self.empty_installments_df = pd.DataFrame(
                        columns=['fecha_de_cargo' ,
                                'descripcion', 
                                'categoria', 
                                'monto', 
                                'forma_de_pago', 
                                'realizado_por', 
                                'limite_de_pago', 
                                'deuda_a_plazos_recurrente_o_normal', 
                                'comentarios'
                                ])

        self.tdc_df = pd.read_excel(excel_dbfs_path, engine = "openpyxl", sheet_name = "tarjetas_de_credito")
        self.condition_blank_comment = lambda text: text if text == text else ""
        self.df = df
        
        
    def get_pay_deadlines(self, row, date_range_i_row):
        """
        :ref:`row` is the current iteration row
        """
        try:
            dia_de_corte = self.tdc_df.loc[tdc_df["credit_cards"] == row["forma_de_pago"]]["dia_de_corte"].values[0]
            plazo_para_pagar = self.tdc_df.loc[tdc_df["credit_cards"] == row["forma_de_pago"]]["plazo_para_pagar"].values[0]
            return [
                pd.to_datetime(f"{date.year}-{date.month}-{dia_de_corte}", format = "%Y-%m-%d") + pd.DateOffset(days= plazo_para_pagar, months=1) 
                if date.day > dia_de_corte 
                else pd.to_datetime(f"{date.year}-{date.month}-{dia_de_corte}", format = "%Y-%m-%d") + pd.DateOffset(days= plazo_para_pagar) 
                for date in date_range_i_row
            ]
        except IndexError:
            return date_range_i_row


    def create_nthly_installments(self,row, date_range_i_row):
        return  pd.DataFrame(
            data={'fecha_de_cargo': date_range_i_row,
                    'descripcion': self.condition_blank_comment(row["descripcion"]), 
                    'categoria': row["categoria"], 
                    'monto': row["monto_total_deuda"] / row["numero_de_plazos"], 
                    'forma_de_pago': row["forma_de_pago"], 
                    'realizado_por': row["realizado_por"], 
                    'limite_de_pago': self.get_pay_deadlines(row, date_range_i_row), 
                    'deuda_a_plazos_recurrente_o_normal': "Plazos", 
                    'comentarios': self.condition_blank_comment(row["comentarios"])
                    }
                )

    
    def create_monthly_date_range(self, row):
        return  pd.date_range(
                    start = f"{row['fecha_de_inicio'].year}-{row['fecha_de_inicio'].month}-01", 
                    periods = row["numero_de_plazos"], 
                    freq = "MS"
                ) + pd.DateOffset(days=row['fecha_de_inicio'].day - 1)

    def create_monthly_installments(self, frequency_desc_column: str = "frecuencia_de_pago", frecuency_desc_value: str = "mensual"):
        df = self.df.loc[self.df[frequency_desc_column] == frecuency_desc_value].copy()
        if len(df) > 0:
            df_new_installments = self.empty_installments_df
            for _,row in df.iterrows():
                date_range_i_row = self.create_monthly_date_range(row)
                if len(df_new_installments) == 0:
                    df_new_installments = self.create_nthly_installments(row, date_range_i_row)
                else:
                    df_new_installments = (
                        pd.concat([
                            df_new_installments,
                            self.create_nthly_installments(row, date_range_i_row)
                        ],
                            ignore_index=True
                        )
                    ) 
            print("Rows generated successfully")
            return df_new_installments
        else:
            return None
    
        
    def create_semimonthly_date_range(self, row):
        return pd.date_range(
                    start = f"{row['fecha_de_inicio']}", 
                    periods = row["numero_de_plazos"], 
                    freq = "SME"
                ) + pd.DateOffset(days=row["dias_de_margen"])
    
    def create_semimonthly_installments(self, frequency_desc_column: str = "frecuencia_de_pago", frecuency_desc_value: str = "quincenal"):
        df = self.df.loc[self.df[frequency_desc_column] == frecuency_desc_value].copy()
        if len(df) > 0:
            df_new_installments = self.empty_installments_df
            for _,row in df.iterrows():
                date_range_i_row = self.create_semimonthly_date_range(row)
                if len(df_new_installments) == 0:
                    df_new_installments = self.create_nthly_installments(row, date_range_i_row)
                else:
                    df_new_installments = (
                        pd.concat([
                            df_new_installments,
                            self.create_nthly_installments(row, date_range_i_row)
                        ],
                            ignore_index=True
                        )
                    ) 
            print("Rows generated successfully")
            return df_new_installments
        else:
            return None
        
    def create_bimonthly_date_range(self, row):
        return pd.date_range(
                start = pd.to_datetime(f"{row['fecha_de_inicio'].year}-{row['fecha_de_inicio'].month}-01", format="%Y-%m-%d"), 
                periods = row['numero_de_plazos'], 
                freq = "2MS"
                ) + pd.DateOffset(days=row['fecha_de_inicio'].day - 1)
    
    def create_bimonthly_installments(self, frequency_desc_column: str = "frecuencia_de_pago", frecuency_desc_value: str = "bimestral"):
        df = self.df.loc[self.df[frequency_desc_column] == frecuency_desc_value].copy()
        if len(df) > 0:
            df_new_installments = self.empty_installments_df
            for _,row in df.iterrows():
                date_range_i_row = self.create_bimonthly_date_range(row)
                if len(df_new_installments) == 0:
                    df_new_installments = self.create_nthly_installments(row, date_range_i_row)
                else:
                    df_new_installments = (
                        pd.concat([
                            df_new_installments,
                            self.create_nthly_installments(row, date_range_i_row)
                        ],
                            ignore_index=True
                        )
                    ) 
            print("Rows generated successfully")
            return df_new_installments
        else:
            return None
        
    def create_weekly_date_range(self, row):
        return pd.date_range(start = row['fecha_de_inicio'], periods = row['numero_de_plazos'], freq = "7D") + pd.DateOffset(days=row['dias_de_margen'])
    
    def create_weekly_installments(self, frequency_desc_column: str = "frecuencia_de_pago", frecuency_desc_value: str = "semanal"):
        df = self.df.loc[self.df[frequency_desc_column] == frecuency_desc_value].copy()
        if len(df) > 0:
            df_new_installments = self.empty_installments_df
            for _,row in df.iterrows():
                date_range_i_row = self.create_weekly_date_range(row)
                if len(df_new_installments) == 0:
                    df_new_installments = self.create_nthly_installments(row, date_range_i_row)
                else:
                    df_new_installments = (
                        pd.concat([
                            df_new_installments,
                            self.create_nthly_installments(row, date_range_i_row)
                        ],
                            ignore_index=True
                        )
                    ) 
            print("Rows generated successfully")
            return df_new_installments
        else:
            return None
        
    def create_biweekly_date_range(self, row):
        return pd.date_range(start = row['fecha_de_inicio'], periods = row['numero_de_plazos'], freq = "14D") + pd.DateOffset(days=row['dias_de_margen'])
    
    
    def create_biweekly_installments(self, frequency_desc_column: str = "frecuencia_de_pago", frecuency_desc_value: str = "catorcenal"):
        df = self.df.loc[self.df[frequency_desc_column] == frecuency_desc_value].copy()
        if len(df) > 0:
            df_new_installments = self.empty_installments_df
            for _,row in df.iterrows():
                date_range_i_row = self.create_weekly_date_range(row)
                if len(df_new_installments) == 0:
                    df_new_installments = self.create_nthly_installments(row, date_range_i_row)
                else:
                    df_new_installments = (
                        pd.concat([
                            df_new_installments,
                            self.create_nthly_installments(row, date_range_i_row)
                        ],
                            ignore_index=True
                        )
                    ) 
            print("Rows generated successfully")
            return df_new_installments
        else:
            return None
        
    def create_fourmonthly_date_range(self, row):
        return pd.date_range(
        start = pd.to_datetime(f"{row['fecha_de_inicio'].year}-{row['fecha_de_inicio'].month}-01", format="%Y-%m-%d"), 
        periods = row['numero_de_plazos'], 
        freq = "4MS"
        ) + pd.DateOffset(days=row['fecha_de_inicio'].day - 1)
    
    
    def create_fourmonthly_installments(self, frequency_desc_column: str = "frecuencia_de_pago", frecuency_desc_value: str = "tetramestral"):
        df = self.df.loc[self.df[frequency_desc_column] == frecuency_desc_value].copy()
        if len(df) > 0:
            df_new_installments = self.empty_installments_df
            for _,row in df.iterrows():
                date_range_i_row = self.create_fourmonthly_date_range(row)
                if len(df_new_installments) == 0:
                    df_new_installments = self.create_nthly_installments(row, date_range_i_row)
                else:
                    df_new_installments = (
                        pd.concat([
                            df_new_installments,
                            self.create_nthly_installments(row, date_range_i_row)
                        ],
                            ignore_index=True
                        )
                    ) 
            print("Rows generated successfully")
            return df_new_installments
        else:
            return None
        
    def create_quarterly_date_range(self, row):
        return pd.date_range(
        start = pd.to_datetime(f"{row['fecha_de_inicio'].year}-{row['fecha_de_inicio'].month}-01", format="%Y-%m-%d"), 
        periods = row['numero_de_plazos'], 
        freq = "3MS"
        ) + pd.DateOffset(days=row['fecha_de_inicio'].day - 1)
    
    def create_quarterly_installments(self, frequency_desc_column: str = "frecuencia_de_pago", frecuency_desc_value: str = "trimestral"):
        df = self.df.loc[self.df[frequency_desc_column] == frecuency_desc_value].copy()
        if len(df) > 0:
            df_new_installments = self.empty_installments_df
            for _,row in df.iterrows():
                date_range_i_row = self.create_quarterly_date_range(row)
                if len(df_new_installments) == 0:
                    df_new_installments = self.create_nthly_installments(row, date_range_i_row)
                else:
                    df_new_installments = (
                        pd.concat([
                            df_new_installments,
                            self.create_nthly_installments(row, date_range_i_row)
                        ],
                            ignore_index=True
                        )
                    ) 
            print("Rows generated successfully")
            return df_new_installments
        else:
            return None    
    
    def create_semiyearly_date_range(self, row):
        return pd.date_range(
        start = pd.to_datetime(f"{row['fecha_de_inicio'].year}-{row['fecha_de_inicio'].month}-01", format="%Y-%m-%d"), 
        periods = row['numero_de_plazos'], 
        freq = "6MS"
        ) + pd.DateOffset(days=row['fecha_de_inicio'].day - 1)
    
    def create_semiyearly_installments(self, frequency_desc_column: str = "frecuencia_de_pago", frecuency_desc_value: str = "semestral"):
        df = self.df.loc[self.df[frequency_desc_column] == frecuency_desc_value].copy()
        if len(df) > 0:
            df_new_installments = self.empty_installments_df
            for _,row in df.iterrows():
                date_range_i_row = self.create_semiyearly_date_range(row)
                if len(df_new_installments) == 0:
                    df_new_installments = self.create_nthly_installments(row, date_range_i_row)
                else:
                    df_new_installments = (
                        pd.concat([
                            df_new_installments,
                            self.create_nthly_installments(row, date_range_i_row)
                        ],
                            ignore_index=True
                        )
                    ) 
            print("Rows generated successfully")
            return df_new_installments
        else:
            return None
        
    def create_annual_date_range(self, row):
        return pd.date_range(
        start = pd.to_datetime(f"{row['fecha_de_inicio'].year}-{row['fecha_de_inicio'].month}-01", format="%Y-%m-%d"), 
        periods = row['numero_de_plazos'], 
        freq = "YS"
        ) + pd.DateOffset(months=row['fecha_de_inicio'].month - 1,  days=row['fecha_de_inicio'].day - 1)
        
    def create_annual_installments(self, frequency_desc_column: str = "frecuencia_de_pago", frecuency_desc_value: str = "anual"):
        df = self.df.loc[self.df[frequency_desc_column] == frecuency_desc_value].copy()
        if len(df) > 0:
            df_new_installments = self.empty_installments_df
            for _,row in df.iterrows():
                date_range_i_row = self.create_annual_date_range(row)
                if len(df_new_installments) == 0:
                    df_new_installments = self.create_nthly_installments(row, date_range_i_row)
                else:
                    df_new_installments = (
                        pd.concat([
                            df_new_installments,
                            self.create_nthly_installments(row, date_range_i_row)
                        ],
                            ignore_index=True
                        )
                    ) 
            print("Rows generated successfully")
            return df_new_installments
        else:
            return None
        
    def pdconcat_wise(self, base_df: pd.DataFrame, concat_df: pd.DataFrame):
        if concat_df is None:
            return base_df
        elif len(base_df) > 0:
            return pd.concat(
                        [
                            base_df,
                            concat_df
                        ],
                            ignore_index=True
                    )
        else:
            return concat_df

    def create_all_installments(self):
        df_installments_cum = self.empty_installments_df
        df_installments_cum = self.pdconcat_wise(df_installments_cum, self.create_weekly_installments())
        df_installments_cum = self.pdconcat_wise(df_installments_cum, self.create_biweekly_installments())
        df_installments_cum = self.pdconcat_wise(df_installments_cum, self.create_semimonthly_installments())
        df_installments_cum = self.pdconcat_wise(df_installments_cum, self.create_monthly_installments())
        df_installments_cum = self.pdconcat_wise(df_installments_cum, self.create_bimonthly_installments()) 
        df_installments_cum = self.pdconcat_wise(df_installments_cum, self.create_quarterly_installments())
        df_installments_cum = self.pdconcat_wise(df_installments_cum, self.create_fourmonthly_installments())
        df_installments_cum = self.pdconcat_wise(df_installments_cum, self.create_semiyearly_installments())
        df_installments_cum = self.pdconcat_wise(df_installments_cum, self.create_annual_installments())
        return df_installments_cum

In [43]:
installments_creator = InstallmentsCreator(dfp)

In [46]:
df_all_installments = installments_creator.create_all_installments()

Rows generated successfully
Rows generated successfully
Rows generated successfully
Rows generated successfully
Rows generated successfully


In [47]:
%store df_all_installments

Stored 'df_all_installments' (DataFrame)
